# How can I get variables from different tables with 'require_all_on' [#265](https://github.com/intake/intake-esm/issues/265)
conda env: `phd_v3`

In [1]:
!conda list intake

# packages in environment at /home/lalandmi/miniconda3/envs/phd_v3:
#
# Name                    Version                   Build  Channel
intake                    0.6.0                      py_0    conda-forge
intake-esm                2020.6.11                  py_0    conda-forge


In [2]:
import intake

In [3]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
print(type(col))
col

/home/lalandmi/miniconda3/envs/phd_v3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'intake_esm.core.esm_datastore'>


,unique
activity_id,15
institution_id,33
source_id,76
experiment_id,105
member_id,204
table_id,29
variable_id,373
grid_label,10
zstore,282265
dcpp_init_year,60


In [88]:
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,AerChemMIP,AS-RCEC,TaiESM1,histSST,r1i1p1f1,AERmon,od550aer,gn,gs://cmip6/AerChemMIP/AS-RCEC/TaiESM1/histSST/...,NaN,20200310
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20190718
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20191127
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20190809
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20191127


In [142]:
query = dict(
    experiment_id=['historical'],
    variable_id=['tas', 'snc', 'pr'],
    table_id=['Amon', 'LImon'],
    source_id=['BCC-CSM2-MR', 'SAM0-UNICON', 'NorESM2-LM']
)

In [143]:
# cat = col.search(require_all_on=["source_id"], **query)
# cat = col.search(require_all_on=["variable_id"], **query)
cat = col.search(**query)

In [144]:
cat.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id", "member_id"]
].nunique()

,experiment_id,variable_id,table_id,member_id
source_id,,,,
BCC-CSM2-MR,1,3,2,3
NorESM2-LM,1,2,1,3
SAM0-UNICON,1,3,2,1


In [145]:
cat.df.groupby("source_id")["table_id"].unique() 

source_id
BCC-CSM2-MR    [Amon, LImon]
NorESM2-LM            [Amon]
SAM0-UNICON    [Amon, LImon]
Name: table_id, dtype: object

In [146]:
cat.df.groupby("source_id")["variable_id"].unique()    

source_id
BCC-CSM2-MR    [pr, tas, snc]
NorESM2-LM          [pr, tas]
SAM0-UNICON    [pr, tas, snc]
Name: variable_id, dtype: object

https://github.com/intake/intake-esm/issues/265#issuecomment-671835364

https://stackoverflow.com/questions/48390150/pandas-select-rows-after-groupby

In [147]:
cat.df = cat.df.groupby("source_id").filter(
    lambda x : pd.Series(['pr', 'snc', 'tas']).isin(x['variable_id']).all()
)

In [148]:
cat.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id", "member_id"]
].nunique()

,experiment_id,variable_id,table_id,member_id
source_id,,,,
BCC-CSM2-MR,1,3,2,3
SAM0-UNICON,1,3,2,1


In [124]:
[key for key in cat.keys()]

['CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn',
 'CMIP.BCC.BCC-CSM2-MR.historical.LImon.gn',
 'CMIP.SNU.SAM0-UNICON.historical.Amon.gn',
 'CMIP.SNU.SAM0-UNICON.historical.LImon.gn']

In [125]:
dset_dict = cat.to_dataset_dict(
    zarr_kwargs={"consolidated": True, "decode_times": True}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [128]:
ds = dset_dict[cat.keys()[0]]
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 160, lon: 320, member_id: 3, time: 1980)
Coordinates:
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(160, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(320, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lon        (lon) float64 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
  * lat        (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
  * member_id  (member_id) <U8 'r1i1p1f1' 'r2i1p1f1' 'r3i1p1f1'
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    pr         (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 600, 160, 320), meta=np.ndarray>
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 600, 160, 320), meta=np.ndarray>
Attributes:
    grid:                    T106
    Conventions:             CF-1.7 CMIP-6.2
    experiment_id:           historical
    contact:                 Dr. Tongwen Wu(twwu@cma.gov.cn)
    history:                 2018-11-26T05:08:26Z ; CMOR rewrote data to be c...
    nominal_resolution:      100 km
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-CSM...
    parent_source_id:        BCC-CSM2-MR
    table_id:                Amon
    creation_date:           2018-11-19T03:26:44Z
    data_specs_version:      01.00.27
    initialization_index:    1
    sub_experiment_id:       none
    parent_mip_era:          CMIP6
    run_variant:             forcing: greenhouse gases,solar constant,aerosol...
    activity_id:             CMIP
    parent_time_units:       days since 1850-01-01
    license:                 CMIP6 model data produced by BCC is licensed und...
    description:             DECK: historical
    sub_experiment:          none
    title:                   BCC-CSM2-MR output prepared for CMIP6
    experiment:              all-forcing simulation of the recent past
    branch_time_in_child:    0.0
    realm:                   atmos
    cmor_version:            3.3.2
    source_id:               BCC-CSM2-MR
    mip_era:                 CMIP6
    physics_index:           1
    variant_label:           r3i1p1f1
    comment:                 The model integration starts from the piControl ...
    branch_method:           Standard
    forcing_index:           1
    source_type:             AOGCM
    tracking_id:             hdl:21.14100/7b6d329a-4b9a-4646-8e7c-0c2a56bfd09...
    references:              Model described by Tongwen Wu et al. (JGR 2013; ...
    table_info:              Creation Date:(30 July 2018) MD5:e53ff52009d0b97...
    frequency:               mon
    institution:             Beijing Climate Center, Beijing 100081, China
    parent_experiment_id:    piControl
    product:                 model-output
    realization_index:       3
    parent_variant_label:    r3i1p1f1
    external_variables:      areacella
    source:                  BCC-CSM 2 MR (2017):   aerosol: none  atmos: BCC...
    parent_activity_id:      CMIP
    institution_id:          BCC
    status:                  2019-10-25;created;by nhn2@columbia.edu
    branch_time_in_parent:   2439.0
    grid_label:              gn
    intake_esm_dataset_key:  CMIP.BCC.BCC-CSM2-MR.historical.Amon.gn

## Solution?
https://github.com/intake/intake-esm/issues/265#issuecomment-671484238

In [11]:
import pandas as pd

In [71]:
q1 = dict(
    experiment_id=['historical'], 
    variable_id=['tas', 'pr'], 
    table_id='Amon',
    source_id=['BCC-CSM2-MR', 'SAM0-UNICON', 'NorESM2-LM'],
    member_id='r1i1p1f1'
)
col_subset = col.search(**q1, require_all_on='source_id')
col_subset

,unique
activity_id,1
institution_id,3
source_id,3
experiment_id,1
member_id,1
table_id,1
variable_id,2
grid_label,1
zstore,6
dcpp_init_year,0


In [72]:
q2 = dict(
    experiment_id=['historical'], 
    variable_id=['snc'], 
    table_id='LImon',
    source_id=['BCC-CSM2-MR', 'SAM0-UNICON', 'NorESM2-LM'],
    member_id='r1i1p1f1'
)
col_subset_2 = col.search(**q2, require_all_on='source_id')
col_subset_2

,unique
activity_id,1
institution_id,2
source_id,2
experiment_id,1
member_id,1
table_id,1
variable_id,1
grid_label,1
zstore,2
dcpp_init_year,0


In [73]:
col_subset.df = pd.concat([col_subset.df, col_subset_2.df])
col_subset

,unique
activity_id,1
institution_id,3
source_id,3
experiment_id,1
member_id,1
table_id,2
variable_id,3
grid_label,1
zstore,8
dcpp_init_year,0


In [74]:
col_subset.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP/BCC/BCC-CSM2-MR/historical/r1i...,NaN,20181126
1,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP/BCC/BCC-CSM2-MR/historical/r1i...,NaN,20181126
2,CMIP,NCC,NorESM2-LM,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP/NCC/NorESM2-LM/historical/r1i1...,NaN,20190815
3,CMIP,NCC,NorESM2-LM,historical,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP/NCC/NorESM2-LM/historical/r1i1...,NaN,20190815
4,CMIP,SNU,SAM0-UNICON,historical,r1i1p1f1,Amon,pr,gn,gs://cmip6/CMIP/SNU/SAM0-UNICON/historical/r1i...,NaN,20190323
5,CMIP,SNU,SAM0-UNICON,historical,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP/SNU/SAM0-UNICON/historical/r1i...,NaN,20190323
0,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,LImon,snc,gn,gs://cmip6/CMIP/BCC/BCC-CSM2-MR/historical/r1i...,NaN,20181114
1,CMIP,SNU,SAM0-UNICON,historical,r1i1p1f1,LImon,snc,gn,gs://cmip6/CMIP/SNU/SAM0-UNICON/historical/r1i...,NaN,20190323


In [75]:
col_subset.unique(['variable_id', 'table_id'])

{'variable_id': {'count': 3, 'values': ['pr', 'snc', 'tas']},
 'table_id': {'count': 2, 'values': ['Amon', 'LImon']}}